# Marvel

In [1]:
import os
import sys

from django.utils import timezone

sys.path.append('/home/ubuntu/anodos.ru/anodos/')
os.environ['DJANGO_SETTINGS_MODULE'] = 'anodos.settings'

from django.core.wsgi import get_wsgi_application
application = get_wsgi_application()

In [13]:
""" Updater.Marvel
    API поставщика имеет странные ограничения на время между запросами.
"""

import time

import catalog.runner
from catalog.models import *
from anodos.models import Log


class Runner(catalog.runner.Runner):

    name  = 'Marvel'
    alias = 'marvel'

    url = 'https://b2b.marvel.ru/Api/'


    def __init__(self):

        super().__init__()

        self.stock_msk = self.take_stock('stock-msk', 'склад в Москве', 3, 10)
        self.stock_spb = self.take_stock('stock-spb', 'склад в Санкт-Петербурге', 3, 10)

        # Дополнительные переменные
        # TODO ??
        self.key = ''
        self.task = {'categories': 'GetCatalogCategories',
                     'catalog': 'GetFullStock',
                     'parameters': 'GetItems',
                     'photos': 'GetItemPhotos'}
        self.request_format = {'xml': '0', 'json': '1'}
        self.cookies = None
        self.categories = {}
        self.currencies = {'RUB': self.rub,
                           'RUR': self.rub,
                           'USD': self.usd,
                           'EUR': self.eur,
                           '': None}
        self.stocks = {'msk' : self.stock_msk, 'spb' : self.stock_spb}

    def run(self):

        # Проверяем наличие параметров авторизации
        if not self.updater.login or not self.updater.password:
            print('Ошибка: Проверьте параметры авторизации. Кажется их нет.')
            return False

        # Загружаем и парсим категирии
        data = self.get_data('categories', 'json')
        self.parse_categories(data)

        # Загружаем и парсим каталог
        data = self.get_data('catalog', 'json')
        self.parse_catalog(data)

    def run_categories(self):

        # Проверяем наличие параметров авторизации
        if not self.updater.login or not self.updater.password:
            print('Ошибка: Проверьте параметры авторизации. Кажется их нет.')
            return False

        data = self.get_data('categories', 'json')
        return data
        
    def run_catalog(self):

        # Проверяем наличие параметров авторизации
        if not self.updater.login or not self.updater.password:
            print('Ошибка: Проверьте параметры авторизации. Кажется их нет.')
            return False

        data = self.get_data('categories', 'json')
        return data

    def get_data(self, task, request_format, pack_status = None, articles = None):

        import requests

        # Создаем сессию
        s = requests.Session()

        # Собираем URL
        if not pack_status is None:
            url = '{url}{task}?user={login}&password={password}&secretKey={key}&packStatus={pack_status}&responseFormat={request_format}'.format(
                url            = self.url,
                task           = self.task[task],
                login          = self.updater.login,
                password       = self.updater.password,
                key            = None,
                pack_status    = pack_status,
                request_format = self.request_format[request_format])
        else:
            url = '{url}{task}?user={login}&password={password}&secretKey={key}&responseFormat={request_format}'.format(
                url            = self.url,
                task           = self.task[task],
                login          = self.updater.login,
                password       = self.updater.password,
                key            = None,
                request_format = self.request_format[request_format])

        if articles:

            # Готовим начало
            url = '{url}{mid}'.format(
                url = url,
                mid = '&getExtendedItemInfo=1&items={"WareItem": [')

            # Добавляем артикулы
            for article in articles:
                url = '{url}{mid}{article}{end}'.format(
                    url     = url,
                    mid     = '{"ItemId": "',
                    article = article,
                    end     = '"},')

            # Удаляем лишнюю запятую
            url = url[0 : len(url) - 1]

            # Добавляем конец
            url = '{url}{end}'.format(
                url = url,
                end = ']}')

        # Выполняем запрос
        try:
            r = s.post(url, cookies = self.cookies, verify = False, timeout = 300)
        except Exception:
            print('Нет соединения')
            return False
        else:

            # Обрабатываем ответ
            if 'json' == request_format:

                import json

                try:
                    data = json.loads(r.text)
                except Exception:
                    return False

                if data['Header']['Key']: self.key = data['Header']['Key']
                if data['Header']['Code'] != 0:

                    print(data['Header']['Message'])

                    if data['Header']['Message']:
                        Log.objects.add(
                        subject     = "catalog.updater.{}".format(self.updater.alias),
                        channel     = "error",
                        title       = "?",
                        description = data['Header']['Message'])
                    else:
                        Log.objects.add(
                        subject     = "catalog.updater.{}".format(self.updater.alias),
                        channel     = "error",
                        title       = "?",
                        description = "Невнятный ответ сервера")
                    return False
                else:
                    return data['Body']
            else:
                print('Ошибка: используется неподдерживаемый формат.')
                return False

In [16]:
s = Runner()
data_categories = s.run_categories()
data_categories

/usr/local/lib/python3.5/dist-packages/urllib3/connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Вы сможете сгрузить запрошенные данные через 3,12769484 мин.


False

In [15]:
s = Runner()
data_catalog = s.run_catalog()
data_catalog

/usr/local/lib/python3.5/dist-packages/urllib3/connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Вы сможете сгрузить запрошенные данные через 6,97856160333333 мин.


False